# Podjetje, varianta 2

In [1]:
import os
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, create_engine
from sqlalchemy.orm import relationship, backref, declarative_base, sessionmaker

In [3]:
Base = declarative_base()


class Oddelek(Base):
    __tablename__ = 'oddelek'

    id = Column(Integer, primary_key=True)
    naziv = Column(String)

    # Ime razreda lahko podamo kot niz (ker še ni definirano)
    seznam_zaposlenih = relationship('Zaposleni', secondary='povezava', back_populates='seznam_oddelkov')

    def __repr__(self):
        return f"Oddelek[{self.id}, {self.naziv}]"

    
class Zaposleni(Base):
    __tablename__ = 'zaposleni'

    id = Column(Integer, primary_key=True)
    ime = Column(String)
    datum_zaposlitve = Column(DateTime, default=func.now())
    seznam_oddelkov = relationship(Oddelek, secondary='povezava', back_populates='seznam_zaposlenih')

    def __repr__(self):
        return f"Zaposleni[{self.id}, {self.ime}, {self.datum_zaposlitve}]"


class Povezava(Base):
    __tablename__ = 'povezava'

    oddelek_id = Column(Integer, ForeignKey('oddelek.id'), primary_key=True)
    zaposleni_id = Column(Integer, ForeignKey('zaposleni.id'), primary_key=True)

Povežimo se na bazo - če že obstaja, jo bomo najprej pobrisali.

In [4]:
baza = 'podjetje2.db'
if os.path.exists(baza):
    os.remove(baza)

engine = create_engine(f'sqlite:///{baza}', echo=True)

In [5]:
Base.metadata.create_all(engine)
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

2024-05-13 17:49:16,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:49:16,853 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("oddelek")
2024-05-13 17:49:16,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:49:16,856 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("oddelek")
2024-05-13 17:49:16,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:49:16,858 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zaposleni")
2024-05-13 17:49:16,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:49:16,859 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("zaposleni")
2024-05-13 17:49:16,860 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:49:16,861 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("povezava")
2024-05-13 17:49:16,862 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:49:16,862 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("povezava")
2024-05-13 17:49:16,863 INFO sqlalchemy.engine.Engine [raw s

Ustvarimo dva oddelka in tri zaposlene.

In [6]:
IT = Oddelek(naziv="IT")
finance = Oddelek(naziv="Finance")
metka = Zaposleni(ime="Metka")
janez = Zaposleni(ime="Janez")
katka = Zaposleni(ime="Katka")

Dodajmo zaposlene v oddelke in zapišimo v bazo.

In [7]:
katka.seznam_oddelkov.append(finance)
finance.seznam_zaposlenih.append(metka)
janez.seznam_oddelkov.append(IT)

In [8]:
session.add(IT)
session.add(finance)
session.add(janez)
session.add(metka)
session.add(katka)
session.commit()

2024-05-13 17:50:15,210 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:50:15,212 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2024-05-13 17:50:15,213 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ('IT',)
2024-05-13 17:50:15,214 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2024-05-13 17:50:15,215 INFO sqlalchemy.engine.Engine [cached since 0.002819s ago] ('Finance',)
2024-05-13 17:50:15,217 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve) VALUES (?, CURRENT_TIMESTAMP)
2024-05-13 17:50:15,217 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ('Janez',)
2024-05-13 17:50:15,218 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve) VALUES (?, CURRENT_TIMESTAMP)
2024-05-13 17:50:15,219 INFO sqlalchemy.engine.Engine [cached since 0.001886s ago] ('Katka',)
2024-05-13 17:50:15,219 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve) VALUES (?, CURRENT

Izpišimo Janezove oddelke.

In [9]:
janez.seznam_oddelkov

2024-05-13 17:50:56,162 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:50:56,168 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni 
WHERE zaposleni.id = ?
2024-05-13 17:50:56,169 INFO sqlalchemy.engine.Engine [generated in 0.00093s] (1,)
2024-05-13 17:50:56,172 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek, povezava 
WHERE ? = povezava.zaposleni_id AND oddelek.id = povezava.oddelek_id
2024-05-13 17:50:56,173 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (1,)


[Oddelek[1, IT]]

Izpišimo zaposlene na oddelku Finance.

In [10]:
finance.seznam_zaposlenih

2024-05-13 17:51:07,100 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.id = ?
2024-05-13 17:51:07,102 INFO sqlalchemy.engine.Engine [generated in 0.00134s] (2,)
2024-05-13 17:51:07,104 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni, povezava 
WHERE ? = povezava.oddelek_id AND zaposleni.id = povezava.zaposleni_id
2024-05-13 17:51:07,105 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (2,)


[Zaposleni[2, Katka, 2024-05-13 15:50:15],
 Zaposleni[3, Metka, 2024-05-13 15:50:15]]

Izpišimo zaposlene na oddelku Finance s poizvedbo na oddelkih.

In [11]:
session.query(Oddelek).filter(Oddelek.naziv == 'Finance').one().seznam_zaposlenih

2024-05-13 17:51:42,673 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.naziv = ?
2024-05-13 17:51:42,674 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ('Finance',)


[Zaposleni[2, Katka, 2024-05-13 15:50:15],
 Zaposleni[3, Metka, 2024-05-13 15:50:15]]

Izpišimo zaposlene na oddelku Finance s poizvedbo na zaposlenih.

In [12]:
session.query(Zaposleni).filter(Zaposleni.seznam_oddelkov.any(Oddelek.naziv == 'Finance')).all()

2024-05-13 17:52:34,299 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni 
WHERE EXISTS (SELECT 1 
FROM oddelek, povezava 
WHERE zaposleni.id = povezava.zaposleni_id AND oddelek.id = povezava.oddelek_id AND oddelek.naziv = ?)
2024-05-13 17:52:34,300 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('Finance',)


[Zaposleni[2, Katka, 2024-05-13 15:50:15],
 Zaposleni[3, Metka, 2024-05-13 15:50:15]]